The code will have many drawbacks
1. sender, receiver, subject ignored
2. body text not formatted properly
3. train test set splitted randomly and not stratified
4. folds not created for cross validation
5. Hyperparameter tuning left
6. more NLP features reamining to be used

In [ ]:
import numpy as np
import pandas as pd
import json

with open('../conf/user_info.json', 'r') as file:
    USER_EMAIL_ID = json.load(file)['USER_EMAIL_ID']

In [ ]:
df = pd.read_csv('../data/training_data.csv', sep='~', index_col=0)
condition = df['sender'] == f'{USER_EMAIL_ID}'
df.drop(df[condition].index, inplace=True)
df

In [ ]:
df_reindexed = df.reset_index(drop=True)
df_reindexed.index

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

with open('../data/label_dict.txt', 'r') as file:
    label_str = file.read()

all_labels = json.loads(label_str.replace('\'', '\"'))
all_labels

In [ ]:
import re

label_list = [key for key in all_labels.keys() if re.match('Label_[0-9]', key)]
label_list

In [ ]:
mlb = MultiLabelBinarizer(classes=label_list)
labels_array = [list(st.split(',')) for st in df['labels']]
mlb.fit(label_list)
labels = mlb.transform(labels_array)
type(labels)

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value='')
df_reindexed_imputed = pd.DataFrame(imputer.fit_transform(df_reindexed))

In [ ]:
df_reindexed_imputed

In [ ]:
# import neattext as nt
# import neattext.functions as nfx
#
# corpus = df_reindexed_imputed[4].apply(nfx.remove_stopwords)

In [ ]:
corpus = df_reindexed_imputed[4]
corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

tfidf = TfidfVectorizer(
    max_df=0.8,
    min_df=0.01
)
X_features_body = tfidf.fit_transform(corpus).toarray()
tfidf.inverse_transform(X_features_body)

In [ ]:
X = pd.DataFrame(X_features_body)
y = pd.DataFrame(labels)

print(X.shape,'\n', y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lst = [X_train, X_test, y_train, y_test]
for i in lst:
    print(i.shape, '\n')


In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
binary_rel_clf = BinaryRelevance(classifier=SVC())
binary_rel_clf.fit(X_train, y_train)
br_predictions = binary_rel_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, br_predictions)

In [ ]:
# to check how much the model is overfitting
test_prediction = binary_rel_clf.predict(X_train)
accuracy_score(y_train, test_prediction)

In [ ]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)
knn_predictions = knn_clf.predict(X_test)
accuracy_score(y_test, knn_predictions)

In [ ]:
knn_train_pr = knn_clf.predict(X_train)
accuracy_score(y_train, knn_train_pr)

In [ ]:
mlb.inverse_transform(knn_train_pr)

In [ ]:
[list(st.split(',')) for st in df['labels']]

In [ ]:
x = df['labels'].str.split(',')
def fun(lst):
    for i in lst:
        if i in ['UNREAD', 'INBOX']:
            lst.remove(i)


x.apply(fun)
x.apply(lambda lst: ','.join(lst))

In [ ]:
df['labels'].str.slice(0,4)